<a href="https://colab.research.google.com/github/rudradeep22/BeyondTheBuzz-Assignment1/blob/main/LSTM_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
ps = PorterStemmer()
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path='/content/drive/MyDrive/IMDB Dataset.csv'
data = pd.read_csv(path)
print(len(data))
data.head(5)

50000


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
X = data.head(1000).get('review')
y0 = data.head(1000).get('sentiment')
review_list = []
y = []
for review in X:
    review_list.append(review)
review_list[0]
for sentiment in y0:
    if sentiment == 'positive':
        y.append(1)
    else:
        y.append(0)
y = torch.tensor(y, dtype=torch.int)

In [ ]:
processed_reviews =[]
for review in tqdm(review_list):
    review = review.lower()
    words = word_tokenize(review)
    stop_words = set(stopwords.words('english'))
    filtered_words = [w for w in words if not w in stop_words]
    stemmed_words = []
    for word in words:
        stemmed_words.append(ps.stem(word))
    processed_reviews.append(" ".join(stemmed_words))


100%|██████████| 1000/1000 [00:10<00:00, 98.32it/s]


In [ ]:
class ReviewClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(ReviewClassifier, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.embedding = nn.Embedding(input_size, 100)
        self.lstm = nn.LSTM(100, hidden_size, num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, 1)

    def forward(self, X):
        batch_size = X.size(0)  # Get the dynamic batch size
        hidden_states = torch.zeros(self.num_layers, batch_size, self.hidden_size)
        cell_states = torch.zeros(self.num_layers, batch_size, self.hidden_size)
        embedding = self.embedding(X)
        out, _ = self.lstm(embedding, (hidden_states, cell_states))
        out = out.view(-1, self.hidden_size)
        out = self.linear(out)
        return out




In [ ]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(processed_reviews)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=1234)

In [ ]:
! pip install scipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import scipy

In [ ]:
X_train_tensor = torch.tensor(scipy.sparse.csr_matrix.todense(X_train)).int()
X_test_tensor = torch.tensor(scipy.sparse.csr_matrix.todense(X_test)).int()

In [ ]:
input_size = X_train_tensor.shape[1]
hidden_size = 128
num_layers = 2
learning_rate = 0.01
num_epochs = 10


In [ ]:
model = ReviewClassifier(input_size, hidden_size, num_layers)
print(model)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), learning_rate)

ReviewClassifier(
  (embedding): Embedding(13393, 100)
  (lstm): LSTM(100, 128, num_layers=2, batch_first=True)
  (linear): Linear(in_features=128, out_features=1, bias=True)
)


In [ ]:
for epoch in range(num_epochs):
      optimizer.zero_grad()
      y_pred = model(X_train_tensor)
      loss = criterion(y_pred,y_train)
      loss.backward()
      optimizer.step()
      print(f'Epoch: {epoch}, Loss: {loss}')